In [1]:
#import rasterio.features
#import rasterio.plot as plot
#from rasterio import Affine
#import json

#import numpy as np
#import cv2 as cv2

#import os
#import sys

#import rasterio.features

import os
import sys
import rasterio
import tensorflow as tf
from tensorflow import keras

import WINMOL_Analyzer as Analyzer
from WINMOL_Analyzer import Timer

from Config import Config

# Root directory of the project
ROOT_DIR = os.path.abspath(".//")

sys.path.append(ROOT_DIR)

# Directory whre the trained models are stored
model_dir = os.path.join(ROOT_DIR, "model//")

# Directory where the inpupt UAV-orthomosaics are stored
image_dir = os.path.join(ROOT_DIR, "input//")

# Directory to save the semantic stem map
pred_dir = os.path.join(ROOT_DIR, "pred//")

# Directory to save logs and trained model
output_dir = os.path.join(ROOT_DIR, "output//")

In [2]:
config = Config()
config.display()


Configurations:
IMG_bit                        8
IMG_height                     512
IMG_width                      512
max_distance                   8
min_length                     2.0
n_Channels                     3
num_classes                    1
overlapp_pred                  8
padding                        1000
tile_size                      15
tolerance_angle                7




In [3]:


# Load your pre-trained U-Net model
model_path=os.path.join(model_dir, "UNet_SpecDS_UNet_Mask-RCNN_512_Spruce_2_model_2023-02-27_061925.hdf5")

# Load the model from the HDF5 file
model = keras.models.load_model(model_path, compile=False)

# Summary of the loaded model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 512, 512, 3)]        0         []                            
                                                                                                  
 conv2d_1 (Conv2D)           (None, 512, 512, 64)         1728      ['input_1[0][0]']             
                                                                                                  
 batch_normalization_1 (Bat  (None, 512, 512, 64)         256       ['conv2d_1[0][0]']            
 chNormalization)                                                                                 
                                                                                                  
 dropout (Dropout)           (None, 512, 512, 64)         0         ['batch_normalization_1[0]

In [4]:
img_list=os.listdir(image_dir)
img_name=img_list[0]
print(img_name)
file_name = os.path.splitext(os.path.basename(img_name))[0]

20220209_Bremerhagen_2.tiff


In [5]:
#Predict with resampling per tile

tt = Timer()
tt.start()

img, profile =Analyzer.IO.load_orthomosaic(image_dir+img_name, config)

pred, profile=Analyzer.Pred.predict_with_resampling_per_tile(img, profile, model,config)   
px_size=profile['transform'][0]
bounds=Analyzer.IO.get_bounds_from_profile(profile)
Analyzer.IO.export_stem_map(pred, profile, pred_dir, file_name)        

prediction=Analyzer.IO.add_padding_to_stem_map(pred, config.padding)
segments = Analyzer.Skel.find_segments(prediction, config.min_length/4, px_size, config.padding,verbose=0)
segments = Analyzer.Vec.restore_geoinformation(segments, bounds, px_size, config.padding)

stems = Analyzer.Vec.build_stem_parts(segments, verbose=0)
stems = Analyzer.Vec.connect_stems(stems,config, verbose=0)  
endnodes = Analyzer.Vec.rebuild_endnodes_from_stems(stems)   
stems = Analyzer.Quant.quantify_stems(stems, pred, px_size, bounds,pred_dir+'pred_'+file_name+'.tiff')

Analyzer.IO.stems_to_geojson(stems, output_dir+ os.path.splitext(os.path.basename(img_name))[0])

tt.stop()    


#######################################################
Prediction of the semantic stem map
Resampling tiles while analyzing
1/1 [==============================] - 1s 747ms/step
24  tiles analyzed
Elapsed time: 19.3453 seconds
#######################################################

#######################################################
Skeletonize Image
Elapsed time: 0.5021 seconds
#######################################################

#######################################################
Splitting the skeleton into segments and detecting endnodes
Dense nodes removed:  7
Brachpoints removed:  277
Detected endnodes:  516
Elapsed time: 9.0138 seconds
#######################################################

#######################################################
Find connected segments in the skeleton
Initial length of skeleton:  10227
Number of endnodes 516
Minimum length in pixel:  17
Detected skeleton segments:  97
Elapsed time: 25.3617 seconds
###################################